In [1]:
pip install transformers datasets evaluate rouge_score

  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=d84ba550ed7075970c93db7448452520d8ba038220c20813d382fb3d6659bb05
  Stored in directory: c:\users\manish_bhoge\appdata\local\pip\cache\wheels\9b\3d\39\09558097d3119ca0a4d462df68f22c6f3c1b345ac63a09b86e
Successfully built rouge-score
  Attempting uninstall: datasets
    Found existing installation: datasets 1.18.3
    Uninstalling datasets-1.18.3:
      Successfully uninstalled datasets-1.18.3
Note: you may need to restart the kernel to use updated packages.


In [1]:
from huggingface_hub import notebook_login

In [3]:
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (manager-core).
Your token has been saved to C:\Users\Manish_Bhoge\.cache\huggingface\token
Login successful


In [3]:
from datasets import load_dataset

billsum = load_dataset("billsum", split="ca_test")

Found cached dataset billsum (C:/Users/Manish_Bhoge/.cache/huggingface/datasets/billsum/default/3.0.0/75cf1719d38d6553aa0e0714c393c74579b083ae6e164b2543684e3e92e0c4cc)


In [4]:
billsum = billsum.train_test_split(test_size=0.2)

In [5]:
billsum["train"][0]

{'text': 'The people of the State of California do enact as follows:\n\n\nSECTION 1.\nSection 1197.5 of the Labor Code is amended to read:\n1197.5.\n(a) An employer shall not pay any of its employees at wage rates less than the rates paid to employees of the opposite sex for substantially similar work, when viewed as a composite of skill, effort, and responsibility, and performed under similar working conditions, except where the employer demonstrates:\n(1) The wage differential is based upon one or more of the following factors:\n(A) A seniority system.\n(B) A merit system.\n(C) A system that measures earnings by quantity or quality of production.\n(D) A bona fide factor other than sex, such as education, training, or experience. This factor shall apply only if the employer demonstrates that the factor is not based on or derived from a sex-based differential in compensation, is job related with respect to the position in question, and is consistent with a business necessity. For purpo

In [14]:
billsum

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 989
    })
    test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 248
    })
})

There are two fields that you’ll want to use:

    text: the text of the bill which’ll be the input to the model.
    summary: a condensed version of text which’ll be the model target.

In [15]:
from transformers import AutoTokenizer

checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# Proper Prompting is required as some models are trained to do multiple tasks 
The preprocessing function you want to create needs to:

    Prefix the input with a prompt so T5 knows this is a summarization task. Some models capable of multiple NLP tasks require prompting for specific tasks.
    Use the keyword text_target argument when tokenizing labels.
    Truncate sequences to be no longer than the maximum length set by the max_length parameter.

In [16]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [17]:
tokenized_billsum = billsum.map(preprocess_function, batched=True)

Map:   0%|          | 0/989 [00:00<?, ? examples/s]

TypeError: __call__() missing 1 required positional argument: 'text'